# PIK3R1 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "PIK3R1"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
PDGFRA
PIK3R2
PIK3R3
PIK3CA
CBL
STAT3
ERBB3
IGF1R
ERBB2
EGFR
KIT
PIK3CB
CRK
IRS1
CTNNB1
PIK3CG
PTEN
SRC
IRS2
PIK3CD
SOS1
FGFR1
SHC1
HRAS
PIK3R1
AKT1
LCP2
ABL1
FYN
LCK
GNAQ
FCGR2A
VAV1
AXL
DOK1


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  PIK3R1


Doing t-test comparisons

                                         Comparison       P_Value
0      AC005618.9_transcriptomics_ENSG00000279855.1  4.733451e-09
1      FP671120.4_transcriptomics_ENSG00000275530.1  1.225371e-08
2        HMGN1P10_transcriptomics_ENSG00000244101.1  1.984333e-08
3      AC012494.1_transcriptomics_ENSG00000229494.1  2.847025e-08
4    RP11-192H6.2_transcriptomics_ENSG00000251654.2  2.878552e-08
5        MTND1P29_transcriptomics_ENSG00000237722.1  3.027026e-08
6       RNA5SP336_transcriptomics_ENSG00000201059.1  3.377987e-08
7         MIR4722_transcriptomics_ENSG00000265672.1  3.852006e-08
8       RN7SL655P_transcriptomics_ENSG00000241963.3  5.775029e-08
9         TRBV5-2_transcriptomics_ENSG00000242771.1  2.335379e-07
10   RP11-93K22.6_transcriptomics_ENSG00000250643.1  3.974502e-07
11       IGLV1-62_transcriptomics_ENSG00000253823.1  4.370122e-07
12       IMPDH1P4_transcriptomics_ENSG00000237206.1  5.557672e-07
13   CTD-2566J3.1_transcriptomics_ENSG00000258837.

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,PIK3R1,AC005618.9_transcriptomics_ENSG00000279855.1,False,4.733451e-09
1,Gbm,PIK3R1,FP671120.4_transcriptomics_ENSG00000275530.1,False,1.225371e-08
2,Gbm,PIK3R1,HMGN1P10_transcriptomics_ENSG00000244101.1,False,1.984333e-08
3,Gbm,PIK3R1,AC012494.1_transcriptomics_ENSG00000229494.1,False,2.847025e-08
4,Gbm,PIK3R1,RP11-192H6.2_transcriptomics_ENSG00000251654.2,False,2.878552e-08
5,Gbm,PIK3R1,MTND1P29_transcriptomics_ENSG00000237722.1,False,3.027026e-08
6,Gbm,PIK3R1,RNA5SP336_transcriptomics_ENSG00000201059.1,False,3.377987e-08
7,Gbm,PIK3R1,MIR4722_transcriptomics_ENSG00000265672.1,False,3.852006e-08
8,Gbm,PIK3R1,RN7SL655P_transcriptomics_ENSG00000241963.3,False,5.775029e-08
9,Gbm,PIK3R1,TRBV5-2_transcriptomics_ENSG00000242771.1,False,2.335379e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"